In [1]:
%reload_ext dotenv
%dotenv ../.env

In [7]:
import pandas as pd

In [3]:
from google import genai

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works in a few words"
)
print(response.text)

AI identifies patterns in vast data to learn, predict, and make decisions.


In [9]:
words = [
    # theme
    "Who's in charge?",
    # theme words
    "leadership",
    "manager",
    "head",
    "director",
    "boss",
    "chief",
    "supervisor",
    # unrelated words
    "dare",
    "super",
    "sobs",
    "flower",
    "work",
    "contain",
    "foot",
    "talk",
    "laptop",
    "hinge",
]

In [ ]:
from google import genai
from google.genai.types import EmbedContentConfig

client = genai.Client()
response = client.models.embed_content(
    model="gemini-embedding-001",
    contents=words,
    config=EmbedContentConfig(
        task_type="SEMANTIC_SIMILARITY",
    ),
)
print(response)

sdk_http_response=HttpResponse(
  headers=<dict len=11>
) embeddings=[ContentEmbedding(
  values=[
    -0.027346142,
    0.0026793897,
    0.009290586,
    -0.06483352,
    -0.023764791,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    -0.017375091,
    0.005511236,
    0.017141035,
    -0.06482476,
    -0.025281893,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    -0.02131225,
    9.396808e-05,
    0.013157671,
    -0.06882189,
    -0.014549548,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    -0.019706758,
    0.018299602,
    -0.0064284555,
    -0.05780958,
    -0.01489805,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    -0.013436253,
    -0.0092776865,
    -0.0043770187,
    -0.081565246,
    -0.014135684,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    -0.017404739,
    0.01393829,
    0.009399541,
    -0.07290805,
    -0.029384842,
    <... 3067 more items ...>,
  ]
), 

In [11]:
assert response.embeddings
embeddings: list[list[float]] = []
for emb in response.embeddings:
    assert emb.values is not None
    embeddings.append(emb.values)
pd.DataFrame({"words": words, "embeddings": embeddings})

,words,embeddings
0,Who's in charge?,"[-0.027346142, 0.0026793897, 0.009290586, -0.0..."
1,leadership,"[-0.017375091, 0.005511236, 0.017141035, -0.06..."
2,manager,"[-0.02131225, 9.396808e-05, 0.013157671, -0.06..."
3,head,"[-0.019706758, 0.018299602, -0.0064284555, -0...."
4,director,"[-0.013436253, -0.0092776865, -0.0043770187, -..."
5,boss,"[-0.017404739, 0.01393829, 0.009399541, -0.072..."
6,chief,"[-0.036963362, -0.004243206, 0.009718994, -0.0..."
7,supervisor,"[-0.021258986, 0.0016032985, -0.0039879275, -0..."
8,dare,"[-0.023897188, -0.0026941353, 0.012353851, -0...."
9,super,"[-0.012652085, 0.006199701, -7.257613e-06, -0...."


In [12]:
import numpy as np


def cosine_similarity(vec1: list[float], vec2: list[float]) -> float:
    """Calculate cosine similarity between two 1D numpy arrays."""
    arr1 = np.array(vec1)
    arr2 = np.array(vec2)
    norm1 = np.linalg.norm(arr1)
    norm2 = np.linalg.norm(arr2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return np.dot(arr1, arr2) / (norm1 * norm2)


theme_similarity = [cosine_similarity(e, embeddings[0]) for e in embeddings]
pd.DataFrame(
    {"words": words, "embeddings": embeddings, "theme_similarity": theme_similarity}
).sort_values("theme_similarity", ascending=False)

,words,embeddings,theme_similarity
0,Who's in charge?,"[-0.027346142, 0.0026793897, 0.009290586, -0.0...",1.000000
1,leadership,"[-0.017375091, 0.005511236, 0.017141035, -0.06...",0.896377
7,supervisor,"[-0.021258986, 0.0016032985, -0.0039879275, -0...",0.874751
5,boss,"[-0.017404739, 0.01393829, 0.009399541, -0.072...",0.870197
6,chief,"[-0.036963362, -0.004243206, 0.009718994, -0.0...",0.866416
2,manager,"[-0.02131225, 9.396808e-05, 0.013157671, -0.06...",0.866081
4,director,"[-0.013436253, -0.0092776865, -0.0043770187, -...",0.850086
3,head,"[-0.019706758, 0.018299602, -0.0064284555, -0....",0.830105
15,talk,"[-0.026282128, -0.00398074, 0.01034579, -0.072...",0.816123
12,work,"[-0.013638086, 0.011522909, 0.011527616, -0.06...",0.813642
